In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [21]:
classes = {
    'bro': 0,
    'kro': 1,
    'par': 2,
    'pte': 3,
    'rex': 4, 
    'ste': 5,
    'tri': 6
}

def get_label(filename): 
    image_name = filename.split('\\')[-1]
    return classes[image_name[:3]]

In [32]:
class LabeledDataset(Dataset):
    def __init__(self, root_dir, file_list, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []

        for f in file_list:
            self.samples.append((f, get_label(f)))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_name, label = self.samples[idx]
        image = Image.open(img_name)
        if self.transform:
            image = self.transform(image)
        return image, label

# Define transformations to apply to your images
transform = transforms.Compose([
    # transforms.Resize((200, 200)), # not necessary, since images are already 200x200
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

In [33]:
# perform train test split
# iterate through all subfolders (rex, tri, etc)
current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
images_dir = os.path.join(project_dir, 'augmented_images')

file_list = []

for folder in os.listdir(images_dir):
    if folder == 'mis':
        continue
    folder_fp = os.path.join(images_dir, folder) 
    for img in os.listdir(folder_fp):
        file_list.append(os.path.join(folder_fp, img))

train_indices, val_indices = train_test_split(range(len(file_list)), test_size=0.2, random_state=42)
train_files = [file_list[i] for i in train_indices]
validation_files = [file_list[i] for i in val_indices]

In [34]:
current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)
images_dir = os.path.join(project_dir, 'augmented_images')

training = LabeledDataset(file_list = train_files, root_dir = images_dir, transform=transform)
validation = LabeledDataset(file_list = validation_files, root_dir = images_dir, transform=transform)

# Create a dataloader
batch_size = 32
dataloader = DataLoader(training, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation, batch_size=batch_size, shuffle=True)

In [72]:
class CNNClassifier(nn.Module):
    def __init__(self, num_classes=7):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 4, stride=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        # self.conv3 = nn.Conv2d(16, 32, kernel_size=3, stride=2, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(32 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        # x = self.relu(self.conv3(x))
        # x = self.pool(x)
        # print(x.shape)
        x = x.view(-1, 32 * 8 * 8)  # Flatten the output of the convolutional layers
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [76]:
model = CNNClassifier()
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),
                             lr=1e-4, 
                             weight_decay=1e-6)

In [78]:
# Training loop
num_epochs = 50

for epoch in range(num_epochs):
    training_loss = 0.0
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights
        training_loss += loss.item() * inputs.size(0)

    validation_loss = 0.0
    for inputs, labels in validation_loader:
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, labels)  # Calculate the loss
        validation_loss += loss.item() * inputs.size(0)

        


    training_loss = training_loss / len(training)
    validation_loss = validation_loss / len(validation)

    print(f'Epoch [{epoch+1}/{num_epochs}], traing loss: {training_loss:.5f}, validation loss: {validation_loss:.5f}')

Epoch [1/50], traing loss: 1.30721, validation loss: 1.26830
Epoch [2/50], traing loss: 1.18888, validation loss: 1.14973
Epoch [3/50], traing loss: 1.05345, validation loss: 1.03776
Epoch [4/50], traing loss: 0.93274, validation loss: 0.96154
Epoch [5/50], traing loss: 0.82835, validation loss: 0.87566
Epoch [6/50], traing loss: 0.74781, validation loss: 0.81066
Epoch [7/50], traing loss: 0.67715, validation loss: 0.76041
Epoch [8/50], traing loss: 0.62414, validation loss: 0.72409
Epoch [9/50], traing loss: 0.56461, validation loss: 0.67203
Epoch [10/50], traing loss: 0.52019, validation loss: 0.63049
Epoch [11/50], traing loss: 0.48604, validation loss: 0.62147
Epoch [12/50], traing loss: 0.44999, validation loss: 0.58911
Epoch [13/50], traing loss: 0.41199, validation loss: 0.55788
Epoch [14/50], traing loss: 0.37813, validation loss: 0.56563
Epoch [15/50], traing loss: 0.34385, validation loss: 0.50135
Epoch [16/50], traing loss: 0.31374, validation loss: 0.52257
Epoch [17/50], tr

In [80]:
for inputs, labels in validation_loader:
    inputs, labels = inputs.to(device), labels.to(device)  # Move data to the appropriate device
    outputs = model(inputs)  # Forward pass
    print(outputs)


tensor([[ 4.2054e+00,  3.8510e+00, -5.9064e-01, -3.9113e+00,  1.4726e+01,
         -3.1122e+01, -1.0531e+00],
        [ 7.0209e+00,  6.2791e+00, -8.8953e+00, -4.8897e+01,  1.6773e+01,
         -1.6705e+01, -1.3234e+01],
        [-7.7829e+00,  1.9486e+00, -4.8614e+01, -1.9816e+01, -2.8354e+00,
          2.3081e+01,  8.4569e+00],
        [-3.8200e+00, -4.8417e+00,  1.4484e+01, -5.9709e+00,  3.3886e+00,
         -2.8744e+01, -1.0484e+01],
        [ 3.0799e+00, -3.2849e+00,  1.9009e+01, -1.3452e+01,  9.5474e+00,
         -3.5719e+01, -1.1356e+01],
        [ 6.6770e+00, -4.2767e+00, -8.0442e+00, -2.1728e+01,  5.7701e+00,
         -1.1441e+01, -1.1604e+01],
        [-5.2232e+00, -7.1469e+00, -2.5009e+01,  6.1697e-01, -1.8831e-01,
          1.3913e+01,  4.4163e+00],
        [ 1.4820e+01,  1.1144e+01, -2.5673e+01, -3.7901e+01,  1.6685e+01,
         -1.4189e+01, -8.1593e+00],
        [ 1.4725e+01,  6.7648e+00,  1.7381e+00, -2.9564e+01,  1.2607e+01,
         -1.2885e+01, -2.3192e+01],
        [ 